In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm_notebook
import IPython
from sklearn.preprocessing import StandardScaler
import random

Reading in Data

In [2]:
"""Emily's reading in files"""
data_dir = os.getcwd() + '/data/nfl-big-data-bowl-2024/'
players = pd.read_csv(data_dir + "players.csv")
week_1 = pd.read_csv(data_dir + "tracking_week_1.csv")
week_2 = pd.read_csv(data_dir + "tracking_week_2.csv")
week_3 = pd.read_csv(data_dir + "tracking_week_3.csv")
week_4 = pd.read_csv(data_dir + "tracking_week_4.csv")
week_5 = pd.read_csv(data_dir + "tracking_week_5.csv")
week_6 = pd.read_csv(data_dir + "tracking_week_6.csv")
week_7 = pd.read_csv(data_dir + "tracking_week_7.csv")
week_8 = pd.read_csv(data_dir + "tracking_week_8.csv")
week_9 = pd.read_csv(data_dir + "tracking_week_9.csv")
plays = pd.read_csv(data_dir + "plays.csv")
tackles = pd.read_csv(data_dir + "tackles.csv")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

# join player positioning information onto a week's worth of tracking data 
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')

In [2]:
def load_dataset(dataset_name):
    """
    Download a specific dataset from data directory.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    return pd.read_csv(f"C:\\Users\\mattd\\Documents\\GitHub\\big-data-bowl-2024\\data\\{dataset_name}.csv")

# Read In csvs
plays = load_dataset("plays")
players = load_dataset("players")
week_1 = load_dataset("tracking_week_1")
week_2 = load_dataset("tracking_week_2")
week_3 = load_dataset("tracking_week_3")
week_4 = load_dataset("tracking_week_4")
week_5 = load_dataset("tracking_week_5")
week_6 = load_dataset("tracking_week_6")
week_7 = load_dataset("tracking_week_7")
week_8 = load_dataset("tracking_week_8")
week_9 = load_dataset("tracking_week_9")
tackles = load_dataset("tackles")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

(12187398, 18)

Creating the necessary csv to merge back in with output of our model

In [13]:
weeks_upload= pd.concat(weeks_list)
week_with_ids = weeks_upload.merge(players.loc[:, ['nflId', 'position']], how='inner')
week_sorted = week_with_ids.sort_values(['gameId','playId', 'frameId', 'club', 'position', 'jerseyNumber'])
final_upload_week = week_sorted[['gameId','playId', 'frameId', 'nflId', 'position', 'jerseyNumber']]
#final_upload_week.to_csv('all_ids_per_frame.csv', index=False)


11657338

Creating df with input and output data

In [4]:
# get distance from ball
ball = week.groupby(['gameId','playId', 'frameId']).apply(lambda g: g[g['club'] == 'football'])[['x','y']]
week_and_ball = week.merge(ball, on = ['gameId','playId', 'frameId'], how = 'left', suffixes=('','_ball') )
week_and_ball['distance_to_ball'] = np.sqrt((week_and_ball["x"] - week_and_ball["x_ball"]) ** 2 + (week_and_ball["y"] - week_and_ball["y_ball"]) ** 2)
print('1')


#sort by position and jerseyNumber
week_and_ball = week_and_ball.sort_values(['gameId','playId', 'frameId', 'club', 'position', 'jerseyNumber'])

#merge with tackles
input = week_and_ball.copy()
input = input.merge(tackles[['gameId', 'playId', 'nflId', 'tackle', 'assist']], on = ['gameId', 'playId', 'nflId'], how = 'left')
input['tackle'] = input['tackle'].fillna(0)
input['assist'] = input['assist'].fillna(0)

print('2')


#get order of players (0-10)
input['number'] = input.groupby(['gameId','playId', 'frameId', 'club']).cumcount()
input = input[input['club'] != 'football']
# check if needed
input = input.sort_values(by = ['gameId', 'playId', 'frameId', 'position', 'jerseyNumber'])
input = input.reset_index(drop = True)

# merge with plays
input_transform = input.copy()
#check
input_transform = input_transform.reset_index(drop = True)
plays = plays[['gameId', 'playId', 'possessionTeam', 'defensiveTeam']]
all_players = input.merge(plays, on = ['gameId', 'playId'])



print('3')

# offense
offense = all_players[all_players['possessionTeam'] == all_players['club']]
offense = offense.rename(columns={c: 'o_'+c for c in offense.columns if c in ['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_to_ball']})

o_table = pd.pivot_table(offense, values=['o_x', 'o_y', 'o_s', 'o_a', 'o_dis', 'o_o', 'o_dir', 'o_distance_to_ball'], index=['gameId', 'playId', 'frameId'],
                                                                     #,'position', 'jerseyNumber'],
                       columns=['number'], aggfunc="mean")
o_table.columns = o_table.columns.map('{0[0]}_{0[1]}'.format) 
o_table = o_table.reset_index(['gameId','playId','frameId'])

# defense - includes output data
defense = all_players[all_players['defensiveTeam'] == all_players['club']]
defense['nfl_ids'] = defensedefense.groupby(['gameId', 'playId'])['nflId'].unique()

defense = defense.rename(columns={c: 'd_'+c for c in defense.columns if c in ['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_to_ball']})

d_table = pd.pivot_table(defense, values=['d_x', 'd_y', 'd_s', 'd_a', 'd_dis', 'd_o', 'd_dir', 'd_distance_to_ball', 'tackle', 'assist'], index=['gameId', 'playId', 'frameId'],
                                                                     #,'position', 'jerseyNumber'],
                       columns=['number'], aggfunc="mean")
d_table.columns = d_table.columns.map('{0[0]}_{0[1]}'.format) 
d_table = d_table.reset_index(['gameId','playId','frameId'])

# combinate offense and defense
input_output_df = o_table.merge(d_table, on = ['gameId', 'playId', 'frameId'], how = 'right')

# calculate tackle credit 
tackle_cols = ['tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10']
assist_cols = ['assist_0', 'assist_1', 'assist_2', 'assist_3', 'assist_4', 'assist_5', 'assist_6', 'assist_7', 'assist_8', 'assist_9', 'assist_10']
tackle_assist_cols = tackle_cols + assist_cols

input_output_df['total_tacklers'] = input_output_df[tackle_assist_cols].sum(axis=1)
input_output_df['tackle_11'] = np.where(input_output_df['total_tacklers'] == 0 , 1, 0)

input_output_df['tackle_0'] = input_output_df['tackle_0'] + input_output_df['assist_0']
input_output_df['tackle_1'] = input_output_df['tackle_1'] + input_output_df['assist_1']
input_output_df['tackle_2'] = input_output_df['tackle_2'] + input_output_df['assist_2']
input_output_df['tackle_3'] = input_output_df['tackle_3'] + input_output_df['assist_3']
input_output_df['tackle_4'] = input_output_df['tackle_4'] + input_output_df['assist_4']
input_output_df['tackle_5'] = input_output_df['tackle_5'] + input_output_df['assist_5']
input_output_df['tackle_6'] = input_output_df['tackle_6'] + input_output_df['assist_6']
input_output_df['tackle_7'] = input_output_df['tackle_7'] + input_output_df['assist_7']
input_output_df['tackle_8'] = input_output_df['tackle_8'] + input_output_df['assist_8']
input_output_df['tackle_9'] = input_output_df['tackle_9'] + input_output_df['assist_9']
input_output_df['tackle_10'] = input_output_df['tackle_10'] + input_output_df['assist_10']

input_output_df.loc[:, tackle_cols] = input_output_df.loc[:, tackle_cols].div(input_output_df['total_tacklers'], axis=0).fillna(0)
# drop unnecessary columns
input_output_df.drop(assist_cols, axis=1, inplace=True)
input_output_df.drop('total_tacklers', axis=1, inplace=True)

1
2
3


In [6]:
just_input = input_output_df.iloc[:, 3:-12]
just_input

,o_a_0,o_a_1,o_a_2,o_a_3,o_a_4,o_a_5,o_a_6,o_a_7,o_a_8,o_a_9,...,d_y_1,d_y_2,d_y_3,d_y_4,d_y_5,d_y_6,d_y_7,d_y_8,d_y_9,d_y_10
0,1.97,2.46,1.15,1.00,0.33,2.47,2.57,4.49,6.34,0.79,...,40.78,33.78,25.47,31.61,21.15,27.53,33.64,28.50,37.91,20.33
1,1.82,2.53,0.61,1.22,0.66,2.59,2.75,4.09,6.53,0.81,...,40.21,33.65,25.54,31.32,21.26,27.69,33.57,28.69,37.60,20.33
2,1.60,2.73,0.49,1.24,0.92,2.56,2.90,3.72,6.70,0.78,...,39.63,33.52,25.59,30.98,21.44,27.88,33.50,28.90,37.25,20.34
3,1.26,2.74,0.89,1.03,1.37,2.40,2.92,3.42,6.31,0.93,...,39.03,33.39,25.63,30.65,21.67,28.13,33.44,29.14,36.87,20.39
4,0.99,2.94,1.24,0.76,1.63,2.50,2.75,2.89,5.84,0.91,...,38.44,33.26,25.65,30.30,21.95,28.42,33.39,29.38,36.48,20.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529874,1.30,0.94,0.41,4.04,1.30,1.16,1.08,0.40,1.89,2.13,...,9.19,19.39,19.81,38.17,21.37,20.89,17.36,24.32,24.13,18.39
529875,1.54,0.91,0.28,4.24,1.46,1.24,1.14,0.38,2.00,2.56,...,9.21,19.28,19.68,37.61,21.44,20.64,17.50,24.33,23.76,18.38
529876,1.58,0.94,0.27,4.13,1.50,1.25,1.22,0.45,2.18,3.01,...,9.23,19.16,19.60,37.06,21.51,20.40,17.67,24.35,23.42,18.39
529877,1.60,1.01,0.48,3.65,1.61,1.32,1.27,0.61,2.38,2.93,...,9.23,19.13,19.55,36.51,21.60,20.19,17.83,24.38,23.11,18.40


In [7]:
just_output = input_output_df.iloc[:, -12:]
just_output

,tackle_0,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
529874,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529875,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529876,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529877,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
scaler = StandardScaler()
normalized_input = pd.DataFrame(scaler.fit_transform(just_input), columns=just_input.columns)
normalized_input

,o_a_0,o_a_1,o_a_2,o_a_3,o_a_4,o_a_5,o_a_6,o_a_7,o_a_8,o_a_9,...,d_y_1,d_y_2,d_y_3,d_y_4,d_y_5,d_y_6,d_y_7,d_y_8,d_y_9,d_y_10
0,0.982338,1.348459,-0.137390,-0.446534,-1.127424,1.018461,1.568344,2.197760,3.364562,-0.911863,...,1.073038,0.654039,-0.194692,0.641409,-0.713721,0.072564,0.791930,0.196206,1.261203,-0.695015
1,0.790808,1.428394,-0.733708,-0.272019,-0.888456,1.133029,1.780948,1.872814,3.509933,-0.897046,...,1.029438,0.641896,-0.184856,0.602057,-0.699989,0.091490,0.783622,0.219676,1.225649,-0.695015
2,0.509898,1.656781,-0.866224,-0.256154,-0.700178,1.104387,1.958118,1.572239,3.640001,-0.919271,...,0.985073,0.629752,-0.177829,0.555919,-0.677520,0.113963,0.775315,0.245616,1.185508,-0.693955
3,0.075763,1.668201,-0.424506,-0.422736,-0.374313,0.951629,1.981741,1.328530,3.341609,-0.808145,...,0.939178,0.617608,-0.172208,0.511139,-0.648808,0.143534,0.768194,0.275263,1.141926,-0.688655
4,-0.268991,1.896588,-0.038003,-0.636913,-0.186035,1.047103,1.780948,0.897977,2.982008,-0.822962,...,0.894048,0.605465,-0.169398,0.463644,-0.613856,0.177837,0.762260,0.304909,1.097197,-0.682296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529874,0.126838,-0.387284,-0.954567,1.964937,-0.425003,-0.232250,-0.191546,-1.124810,-0.040168,0.080858,...,-1.343328,-0.690168,-0.990066,1.531593,-0.686258,-0.712838,-1.140181,-0.320132,-0.319215,-0.900654
529875,0.433286,-0.421542,-1.098125,2.123586,-0.309140,-0.155870,-0.120678,-1.141057,0.043994,0.399418,...,-1.341798,-0.700444,-1.008334,1.455602,-0.677520,-0.742409,-1.123566,-0.318897,-0.361650,-0.901714
529876,0.484360,-0.387284,-1.109168,2.036329,-0.280174,-0.146323,-0.026188,-1.084192,0.181713,0.732794,...,-1.340268,-0.711653,-1.019576,1.380967,-0.668781,-0.770797,-1.103390,-0.316426,-0.400644,-0.900654
529877,0.509898,-0.307349,-0.877266,1.655570,-0.200518,-0.079491,0.032869,-0.954213,0.334735,0.673527,...,-1.340268,-0.714456,-1.026603,1.306333,-0.657547,-0.795637,-1.084402,-0.312720,-0.436198,-0.899594


In [9]:
#concat 
normalized_input_output_df = pd.concat([input_output_df[['gameId', 'playId', 'frameId']], normalized_input, just_output], axis = 1)

In [10]:
# def last_5_converter(df):
#   final_input = pd.DataFrame(columns=['four', 'three', 'two', 'one', 'current', 'tackle_1','tackle_2','tackle_3','tackle_4','tackle_5','tackle_6','tackle_7','tackle_8','tackle_9','tackle_10','tackle_11', 'tackle_12'])
#   for idx, row in df.iterrows():
#     if row['frameId'] < 5:
#       current = df.iloc[idx, 4:-12].tolist()
#       previous_frames = [np.zeros(17) for _ in range(4)]
  #     #print("FrameID",row['frameId'])
  #     # Use a loop to assign the lists
  #     for i in range(1, int(row['frameId'])):
  #         #print(3-i)
  #         previous_frames[4-i] = df.iloc[idx - i, 4:-12].tolist()

  #     new_row = [previous_frames[0], previous_frames[1], previous_frames[2], previous_frames[3], current, df.at[idx,'tackle_0'], df.at[idx,'tackle_1'], df.at[idx,'tackle_2'], df.at[idx,'tackle_3'], df.at[idx,'tackle_4'], df.at[idx,'tackle_5'], df.at[idx,'tackle_6'], df.at[idx,'tackle_7'], df.at[idx,'tackle_8'], df.at[idx,'tackle_9'], df.at[idx,'tackle_10'], df.at[idx, 'tackle_11']]
  #     final_input.loc[len(final_input.index)] = new_row 
  #   else:
  #     current = df.iloc[idx, 4:-12].tolist()
  #     one = df.iloc[idx - 1, 4:-12].tolist()
  #     two = df.iloc[idx - 2, 4:-12].tolist()
  #     three = df.iloc[idx - 3, 4:-12].tolist()
  #     four = df.iloc[idx - 4, 4:-12].tolist()
  #     new_row = [four, three, two, one, current, df.at[idx,'tackle_0'], df.at[idx,'tackle_1'], df.at[idx,'tackle_2'], df.at[idx,'tackle_3'], df.at[idx,'tackle_4'], df.at[idx,'tackle_5'], df.at[idx,'tackle_6'], df.at[idx,'tackle_7'], df.at[idx,'tackle_8'], df.at[idx,'tackle_9'], df.at[idx,'tackle_10'], df.at[idx, 'tackle_11']]
  #     # Assuming final_input.columns contains the correct column names
  #     new_row_series = pd.Series(new_row, index=final_input.columns)
  #     # Concatenate along axis=1
  #     final_input.loc[len(final_input.index)] = new_row 
  #     #final_input = pd.concat([final_input,pd.Series(new_row, index=final_input.columns)], axis = 1, ignore_index=True)
  # return final_input

In [11]:
def last_5_converter(df):
    rows = []
    for idx, row in tqdm_notebook(df.iterrows()):
        current = df.iloc[idx, 3:-12].tolist()
        tackles = [df.at[idx, f'tackle_{i}'] for i in range(12)]
        #ids = [df.at[idx, f'{i}'] for i in ['gameId', 'playId', 'frameId']]
        if row['frameId'] < 5:
            previous_frames = [np.zeros(175) for _ in range(4)]
            for i in range(1, int(row['frameId'])):
                previous_frames[4-i] = df.iloc[idx - i, 3:-12].tolist()
        else:
            previous_frames = [df.iloc[idx - i, 3:-12].tolist() for i in range(1, 5)]
        new_row = previous_frames + [current] + tackles
        rows.append(new_row)
    final_input = pd.DataFrame(rows, columns=['four', 'three', 'two', 'one', 'current', 'tackle_1','tackle_2','tackle_3','tackle_4','tackle_5','tackle_6','tackle_7','tackle_8','tackle_9','tackle_10','tackle_11', 'tackle_12'])
    return final_input

In [13]:
model_input_no_id = last_5_converter(normalized_input_output_df)

0it [00:00, ?it/s]

In [18]:
model_input_no_id

,four,three,two,one,current,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.5098976990321165, 1.656781305528464, -0.866...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.5098976990321165, 1.656781305528464, -0.866...","[0.07576294584065033, 1.668200665418835, -0.42...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,"[0.07576294584065033, 1.668200665418835, -0.42...","[0.5098976990321165, 1.656781305528464, -0.866...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.9823384598581234, 1.3484585884884548, -0.13...","[-0.26899112287021976, 1.8965878632262487, -0....",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529874,"[-0.16684176917811003, -0.30734859561529776, -...","[-0.3839091457738431, -0.0561226780271422, -0....","[-0.4732898302544391, -0.19315499671159056, -0...","[-0.30729713050476093, -0.3301873153960392, -0...","[0.1268376226867052, -0.38728411484789277, -0....",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529875,"[0.1268376226867052, -0.38728411484789277, -0....","[-0.16684176917811003, -0.30734859561529776, -...","[-0.3839091457738431, -0.0561226780271422, -0....","[-0.4732898302544391, -0.19315499671159056, -0...","[0.4332856837630341, -0.4215421945190048, -1.0...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529876,"[0.4332856837630341, -0.4215421945190048, -1.0...","[0.1268376226867052, -0.38728411484789277, -0....","[-0.16684176917811003, -0.30734859561529776, -...","[-0.3839091457738431, -0.0561226780271422, -0....","[0.484360360609089, -0.38728411484789277, -1.1...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529877,"[0.484360360609089, -0.38728411484789277, -1.1...","[0.4332856837630341, -0.4215421945190048, -1.0...","[0.1268376226867052, -0.38728411484789277, -0....","[-0.16684176917811003, -0.30734859561529776, -...","[0.5098976990321165, -0.30734859561529776, -0....",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
model_input = pd.concat([input_output_df[['gameId', 'playId', 'frameId']], model_input_no_id], axis = 1)

In [16]:
model_input

,gameId,playId,frameId,four,three,two,one,current,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12
0,2022090800,56,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2022090800,56,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2022090800,56,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.5098976990321165, 1.656781305528464, -0.866...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2022090800,56,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9823384598581234, 1.3484585884884548, -0.13...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.5098976990321165, 1.656781305528464, -0.866...","[0.07576294584065033, 1.668200665418835, -0.42...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2022090800,56,5,"[0.07576294584065033, 1.668200665418835, -0.42...","[0.5098976990321165, 1.656781305528464, -0.866...","[0.790808421685418, 1.4283941077210496, -0.733...","[0.9823384598581234, 1.3484585884884548, -0.13...","[-0.26899112287021976, 1.8965878632262487, -0....",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529874,2022110700,3787,40,"[-0.16684176917811003, -0.30734859561529776, -...","[-0.3839091457738431, -0.0561226780271422, -0....","[-0.4732898302544391, -0.19315499671159056, -0...","[-0.30729713050476093, -0.3301873153960392, -0...","[0.1268376226867052, -0.38728411484789277, -0....",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529875,2022110700,3787,41,"[0.1268376226867052, -0.38728411484789277, -0....","[-0.16684176917811003, -0.30734859561529776, -...","[-0.3839091457738431, -0.0561226780271422, -0....","[-0.4732898302544391, -0.19315499671159056, -0...","[0.4332856837630341, -0.4215421945190048, -1.0...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529876,2022110700,3787,42,"[0.4332856837630341, -0.4215421945190048, -1.0...","[0.1268376226867052, -0.38728411484789277, -0....","[-0.16684176917811003, -0.30734859561529776, -...","[-0.3839091457738431, -0.0561226780271422, -0....","[0.484360360609089, -0.38728411484789277, -1.1...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529877,2022110700,3787,43,"[0.484360360609089, -0.38728411484789277, -1.1...","[0.4332856837630341, -0.4215421945190048, -1.0...","[0.1268376226867052, -0.38728411484789277, -0....","[-0.16684176917811003, -0.30734859561529776, -...","[0.5098976990321165, -0.30734859561529776, -0....",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [20]:
# average number of frames per play - can work backwards (100k or 200k / 44 to get approx num of plays to grab)
model_input.groupby(['gameId', 'playId'])[['frameId']].agg('count')['frameId'].median()

44.0

In [21]:
random_sample_100k = random.choices(model_input[['gameId', 'playId']].values.tolist(), k=2250)
random_sample_200k = random.choices(model_input[['gameId', 'playId']].values.tolist(), k=5500)

In [22]:
random_sample_100k = [tuple(i) for i in random_sample_100k]
model_input_100k_random_sample = model_input[model_input[['gameId', 'playId']].apply(tuple, axis = 1).isin(random_sample_100k)]

In [23]:
model_input_100k_random_sample

,gameId,playId,frameId,four,three,two,one,current,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12
190,2022090800,167,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7014277372048219, -0.8668972302434624, 0.35...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
191,2022090800,167,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7014277372048219, -0.8668972302434624, 0.35...","[1.1227938211847743, -0.9011553099145746, 0.64...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
192,2022090800,167,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7014277372048219, -0.8668972302434624, 0.35...","[1.1227938211847743, -0.9011553099145746, 0.64...","[1.442010551472617, -0.6270906725456776, 1.044...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
193,2022090800,167,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7014277372048219, -0.8668972302434624, 0.35...","[1.1227938211847743, -0.9011553099145746, 0.64...","[1.442010551472617, -0.6270906725456776, 1.044...","[1.442010551472617, -0.36444539506715135, 1.53...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
194,2022090800,167,5,"[1.442010551472617, -0.36444539506715135, 1.53...","[1.442010551472617, -0.6270906725456776, 1.044...","[1.1227938211847743, -0.9011553099145746, 0.64...","[0.7014277372048219, -0.8668972302434624, 0.35...","[1.4164732130495896, -0.44438091429974624, 1.8...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529830,2022110700,3740,38,"[2.3741234039131176, -0.569993873093824, 0.227...","[2.169824696528898, -0.8668972302434624, 0.094...","[2.0293693352022473, -1.072445708270135, 0.083...","[1.7995332893950005, -0.9696714692567989, -0.0...","[2.578422111297337, -0.569993873093824, 0.4478...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529831,2022110700,3740,39,"[2.578422111297337, -0.569993873093824, 0.4478...","[2.3741234039131176, -0.569993873093824, 0.227...","[2.169824696528898, -0.8668972302434624, 0.094...","[2.0293693352022473, -1.072445708270135, 0.083...","[2.6805714649894465, -0.5243164335323413, 0.58...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529832,2022110700,3740,40,"[2.6805714649894465, -0.5243164335323413, 0.58...","[2.578422111297337, -0.569993873093824, 0.4478...","[2.3741234039131176, -0.569993873093824, 0.227...","[2.169824696528898, -0.8668972302434624, 0.094...","[2.757183480258528, -0.3301873153960392, 0.569...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
529833,2022110700,3740,41,"[2.757183480258528, -0.3301873153960392, 0.569...","[2.6805714649894465, -0.5243164335323413, 0.58...","[2.578422111297337, -0.569993873093824, 0.4478...","[2.3741234039131176, -0.569993873093824, 0.227...","[2.808258157104583, 0.03523220109582355, 0.547...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [24]:
random_sample_200k = [tuple(i) for i in random_sample_200k]
model_input_200k_random_sample = model_input[model_input[['gameId', 'playId']].apply(tuple, axis = 1).isin(random_sample_200k)]

In [25]:
model_input_200k_random_sample

,gameId,playId,frameId,four,three,two,one,current,tackle_1,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11,tackle_12
22,2022090800,80,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4803165591071579, 1.1314907505714114, -0.99...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
23,2022090800,80,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4803165591071579, 1.1314907505714114, -0.99...","[1.2377118440883974, 1.0972326709002995, -0.38...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
24,2022090800,80,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4803165591071579, 1.1314907505714114, -0.99...","[1.2377118440883974, 1.0972326709002995, -0.38...","[0.6248157219357396, 1.2456843494751186, 0.127...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
25,2022090800,80,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4803165591071579, 1.1314907505714114, -0.99...","[1.2377118440883974, 1.0972326709002995, -0.38...","[0.6248157219357396, 1.2456843494751186, 0.127...","[0.4077483453400067, 1.4055553879403082, 0.657...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
26,2022090800,80,5,"[0.4077483453400067, 1.4055553879403082, 0.657...","[0.6248157219357396, 1.2456843494751186, 0.127...","[1.2377118440883974, 1.0972326709002995, -0.38...","[1.4803165591071579, 1.1314907505714114, -0.99...","[0.16514363032124635, 1.9308459428973612, 1.26...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529788,2022110700,3707,54,"[-0.15407309996659632, -0.844058510462721, 0.4...","[-0.10299842312054144, -0.4101228346286341, 0....","[-0.2179164460241649, -0.13605819725973722, 0....","[-0.3966778149853568, 0.01239348131508211, 0.5...","[-0.30729713050476093, -1.118123147831618, 0.4...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
529789,2022110700,3707,55,"[-0.30729713050476093, -1.118123147831618, 0.4...","[-0.15407309996659632, -0.844058510462721, 0.4...","[-0.10299842312054144, -0.4101228346286341, 0....","[-0.2179164460241649, -0.13605819725973722, 0....","[-0.5371331763120075, -1.0952844280508764, 0.5...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
529790,2022110700,3707,56,"[-0.5371331763120075, -1.0952844280508764, 0.5...","[-0.30729713050476093, -1.118123147831618, 0.4...","[-0.15407309996659632, -0.844058510462721, 0.4...","[-0.10299842312054144, -0.4101228346286341, 0....","[-0.690357206850172, -0.9125746698049453, 0.55...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
529791,2022110700,3707,57,"[-0.690357206850172, -0.9125746698049453, 0.55...","[-0.5371331763120075, -1.0952844280508764, 0.5...","[-0.30729713050476093, -1.118123147831618, 0.4...","[-0.15407309996659632, -0.844058510462721, 0.4...","[-0.7925065605422817, -0.7869617110108675, 0.3...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [26]:
model_input_100k_random_sample.to_pickle("model_input_100k_random_sample.pkl")

In [27]:
model_input_200k_random_sample.to_pickle("model_input_200k_random_sample.pkl")